<a href="https://colab.research.google.com/github/jeonbyeongguk/cuckoobee/blob/main/n212a_multiple_regression_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 1 / NOTE 2*

# 📝 Assignment
---

# 다중선형회귀(Multiple Linear Regression)

이번 과제에서도 n211 과제 에서 다루었던 housing price dataset을 가지고 배운 내용을 연습해 보겠습니다.
다음 링크에서 다운로드 받아 사용해 주세요. [House Sales in King County, USA](https://www.kaggle.com/harlfoxem/housesalesprediction)

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.express as px
import plotly.graph_objs as go
import itertools

In [1]:
# 데이터는 캐글에서 받으셔도 되고 이 링크에서 불러와도 됩니다.
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/kc_house_data/kc_house_data.csv')


In [2]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

### 1) 데이터는 May 2014 ~ May 2015 의 2년치 데이터입니다. 2015-01-01 부터 큰 날짜의 데이터는 테스트, 그 이외의 데이터는 훈련 데이터셋으로 나누어 주세요
- **(Urclass Quiz) 훈련데이터 샘플의 갯수를 제출하세요.**

In [3]:
# T를 기준으로 나누고 datetime으로 변환 
def date_ft(x):
    import datetime
    date = x.split('T')[0]
    date = pd.to_datetime(date)
    return date

df['date'] = df['date'].apply(date_ft)

# 날짜 기준으로 정렬하고 index 정렬
df = df.sort_values('date' ,ascending=True).reset_index(drop=True)

In [4]:
train =  df[df['date'] < '2015-01-01']
test =  df[df['date'] >= '2015-01-01'] 
train.shape , test.shape           

((14633, 21), (6980, 21))

### 2) 두 특성(bathrooms, sqft_living)을 사용해서 다중선형회귀 모델을 만들어 보세요
- 학습시 Parameters는 모두 기본값으로 합니다.
- 정규화(normalize) 등 추가적인 데이터처리 없이 진행합니다.
- **(Urclass Quiz) 테스트세트의 MAE 값을 제출하세요.(소수점 이하 반올림)**

In [5]:
# 다중모델 학습을 위한 특성
features = ['bathrooms', 
            'sqft_living']
X_train = train[features]
X_test = test[features]

# label 정의
target = 'price'
y_train = train[target]
y_test = test[target]

from sklearn.linear_model import LinearRegression
# 기준모델로 훈련 에러(MAE) 계산
from sklearn.metrics import mean_absolute_error
model = LinearRegression()

# 모델 fit
model.fit(X_train, y_train)

#훈련 데이터 
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'훈련 에러 MAE: {mae:.0f}')

# 테스트 데이터
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'테스트 에러: {mae:.0f}')

훈련 에러 MAE: 170777
테스트 에러: 179253


### 3) 모델의 회귀계수(coefficients)와 절편(intercept)을 확인해 보세요.
- **(Urclass Quiz) sqft_living의 회귀계수를 제출하세요.(소수점 이하 반올림)**

In [6]:
## 절편(intercept)과 계수들(coefficients)
round(model.coef_[0]),round(model.coef_[1]), round(model.intercept_)

(-5159, 286, -50244)

In [9]:
model.coef_[1].__round__()

286

In [14]:
import plotly.express as px
def surface_3d(df, f1, f2, target, length=20, **kwargs):
    """
    2특성 1타겟 선형모델평면을 시각화 합니다.
    
    df : 데이터프레임
    f1 : 특성 1 열 이름
    f2 : 특성 2 열 이름
    target : 타겟 열 이름
    length : 각 특성의 관측치 갯수
    
    """
    
    # scatter plot(https://plotly.com/python-api-reference/generated/plotly.express.scatter_3d)
    plot = px.scatter_3d(df, x=f1, y=f2, z=target, opacity=0.5, **kwargs)
    
    # 다중선형회귀방정식 학습
    model = LinearRegression()
    model.fit(df[[f1, f2]], df[target])    

    # 좌표축 설정
    x_axis = np.linspace(df[f1].min(), df[f1].max(), length)
    y_axis = np.linspace(df[f2].min(), df[f2].max(), length)
    coords = list(itertools.product(x_axis, y_axis))
    
    # 예측
    pred = model.predict(coords)
    z_axis = pred.reshape(length, length).T
    
    # plot 예측평면
    plot.add_trace(go.Surface(x=x_axis, y=y_axis, z=z_axis, colorscale='Viridis'))
    
    return plot

In [15]:
surface_3d(
    train,
    f1='bathrooms', 
    f2='sqft_living', 
    target='price',  
    title='House Prices'
)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



### 4) MSE, MAE, RMSE, $R^2$ 를 훈련/테스트 데이터에 대해 확인해 보세요
- **(Urclass Quiz) 테스트 데이터에 대한 MSE, MAE, RMSE, $R^2$ 값을 올바르게 적은 보기를 고르세요.(소수점 두자리수 이하 반올림)**

In [18]:
from IPython.display import display
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error ,mean_squared_error
from IPython.display import display, HTML
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# 열 지정 
features = ["bathrooms", "sqft_living"]
x_train = train[features]
x_test = test[features]
# label 지정 
target = "price"
y_train = train[target]
y_test = test[target]

# 훈련 데이터 
y_pred_train = model.predict(x_train)
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_pred_train)

# 테스트 데이터
y_pred_test = model.predict(x_test)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)


display(pd.DataFrame([['MSE', str(round(mse_train, 1)), str(round(mse_test, 1))],
                      ['MAE', str(round(mae_train, 1)), str(round(mae_test, 1))],
                      ['RMSE', str(round(rmse_train, 1)), str(round(rmse_test, 1))],
                      ['R2',str(round(r2_train, 1)), str(round(r2_test, 1))]],
                      columns=['Metric', 'train','test']))

,Metric,train,test
0,MSE,67099053310.0,71083994178.8
1,MAE,170777.3,179252.5
2,RMSE,259034.8,266615.8
3,R2,0.5,0.5


## 🔥도전과제(Github - Discussion)

### 공지로 안내드리는 Github - Discussion 페이지에 올려 보세요:)

### 5)  EDA를 진행하시고, 두개 이상의 특성을 선택하여 특성공학(feature engineering)을 진행한 후 아래의 질문들에 대한 본인만의 리포트를 작성해보시고 서로 논해 보세요. 
- 어떤 이유에서 해당 특성공학을 진행하셨나요? 특성공학을 통해 기대하는 효과는 무엇이었나요? 만약 얻고자 하는 효과를 얻지 못했다면 이유는 무엇이며 어떤 액션을 추가할 수 있을까요? 
- 특성공학을 거친 특성을 포함해 다수의 특성을 사용한 모델을 학습하고 테스트 데이터에 대해 MSE, RMSE, MAE, R2 스코어는 어떠한가요? 해당 결과를 통해 얻을 수 있는 인사이트는 어떤 것이 있었나요? 
- 만들어진 회귀계수를 사용해 모델을 설명해 본다면? 
- 만들어진 모델은 과소적합인가요? 과대적합인가요? 일반화가 잘 된 모델을 만들기 위해 추가로 할 수 있는 액션은 어떤 것이 있을까요? 

※kaggle notebook에서도 힌트를 얻어 보세요. 

## 참고자료
* [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
* ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)
* [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf)
* [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
* [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)